In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
# !pip install --upgrade pip
# !pip install tensorflow 

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [5]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [6]:
def split_df(df):
    df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas

    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      train_dataset = tf.data.Dataset.from_tensor_slices((input, output))
      train_dataset = train_dataset.shuffle(len(input))
      train_dataset = train_dataset.batch(self.batch_size).cache()
      train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (train_dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, random_state=42)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, random_state=42)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)


    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
)])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-23 22:38:35.773459: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 22:38:35.807193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 22:38:35.807237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 22:38:35.808658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 22:38:35.814873: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 22:38:35.815735: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [8]:
from keras import saving
from keras import models

class Tester:
  def __init__(self, run_times=500):
    self.run_times = run_times
    self.better_metrics = []

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    input_25, output_25 = split_df(df_25)
    input_1000, output_1000 = split_df(df_1000)
    trainer.split_dataset(input_25, output_25, input_1000, output_1000)
    return (trainer)

  def Train(self, trainer, epochs=500):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveResults(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def Loop(self, sort_by, boundarie):
    betters = 0

    for i in range(self.run_times):
      print (f"++++++++++++++++++++++++++ {i + 1} ++++++++++++++++++++++++++")
      trainer = self.PrepareTraining()
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{betters}"
        self.SaveModelWeights(model, fileName)
        self.SaveResults(trainer, fileName)
        self.better_metrics.append(metrics)
        betters += 1

  def DisplayBetterResults(self):
    self.better_metrics.sort(key=lambda x: x['mse_sup'])
    df = pd.DataFrame(self.better_metrics)
    display(df)

# Treinando com 25 dados


In [9]:
tester = Tester(run_times=250)
tester.Loop(sort_by='mse_sup', boundarie=2.5)

++++++++++++++++++++++++++ 1 ++++++++++++++++++++++++++


Elapsed time:  12.08621026900073
Stopped at epoch:  415
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 2 ++++++++++++++++++++++++++
Elapsed time:  0.5152444969999124
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 3 ++++++++++++++++++++++++++
Elapsed time:  4.793198950999795
Stopped at epoch:  152
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 4 ++++++++++++++++++++++++++
Elapsed time:  0.4721890259997963
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 5 ++++++++++++++++++++++++++
Elapsed time:  0.480325420000554
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 6 ++++++++++++++++++++++++++
Elapsed time:  0.43799520599986863
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 7 ++++++++++++++++++++++++++
Elapsed time:  0.726

2024-01-23 22:41:36.573586: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  14.47776587899989
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 69 ++++++++++++++++++++++++++
Elapsed time:  0.4234559540000191
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 70 ++++++++++++++++++++++++++
Elapsed time:  0.4478097040000648
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 71 ++++++++++++++++++++++++++
Elapsed time:  2.3900017309997565
Stopped at epoch:  77
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 72 ++++++++++++++++++++++++++
Elapsed time:  0.4458296299999347
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 73 ++++++++++++++++++++++++++
Elapsed time:  3.3437561029995777
Stopped at epoch:  102
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 74 ++++++++++++++++++++++++++
Elapsed time: 

2024-01-23 22:47:55.801416: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  4.848043421000511
Stopped at epoch:  167
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 184 ++++++++++++++++++++++++++
Elapsed time:  0.7790911039992352
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 185 ++++++++++++++++++++++++++
Elapsed time:  0.4083769150001899
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 186 ++++++++++++++++++++++++++
Elapsed time:  0.47909907400025986
Stopped at epoch:  7
1/1 [==============================] - 0s 24ms/step
++++++++++++++++++++++++++ 187 ++++++++++++++++++++++++++
Elapsed time:  0.5988144759994611
Stopped at epoch:  11
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 188 ++++++++++++++++++++++++++
Elapsed time:  6.090292396000223
Stopped at epoch:  213
1/1 [==============================] - 0s 29ms/step
++++++++++++++++++++++++++ 189 ++++++++++++++++++++++++++
Elapse

In [10]:
tester.DisplayBetterResults()

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
0,0.872001,0.705968,-0.390309,0.467022,0.747058,0.537184,1.966197,1.067048,2.258759,1.662904
1,0.892129,0.671060,0.650113,0.604089,0.680741,0.452713,2.199622,0.309274,2.153711,1.231492
2,0.831538,0.657371,0.808470,0.559085,0.740697,0.706998,2.291163,1.519225,2.760687,2.139956
3,0.875486,0.657176,0.653167,0.624895,0.720786,0.522557,2.292465,0.237155,2.283558,1.260357
4,0.867872,0.656636,0.547136,0.651018,0.634715,0.554513,2.296076,0.745019,2.719842,1.732431
5,0.821014,0.651240,0.484773,0.732383,0.718088,0.751166,2.332158,2.448838,2.071010,2.259924
6,0.818899,0.650119,-0.693858,0.764313,0.593583,0.760043,2.339654,1.975068,1.878413,1.926740
7,0.850120,0.647428,-3.881907,0.570674,0.508250,0.629016,2.357652,1.182892,2.608092,1.895492
8,0.916028,0.646178,-1.002180,0.854113,0.719005,0.352411,2.366009,1.456596,0.739586,1.098091
9,0.878021,0.645794,0.707310,0.688968,0.715264,0.511920,2.368574,0.785890,1.804237,1.295063
